# プッシュドキュメントAPIを使用したドキュメントレベルアクセスの例

Azure AI Searchでは、JSONドキュメントペイロードを検索インデックスにアップロードしてインデックス化できます。このノートブックでは、[ドキュメントレベルでのユーザーアクセス権限](azure/search/search-document-level-access-overview)を含むドキュメントをインデックス化し、ユーザーが表示を許可されている結果のみを返すようにインデックスをクエリする方法を示します。

クエリアクセストークンの背後にあるセキュリティプリンシパルが「ユーザー」を決定します。ドキュメント内の権限メタデータは、ユーザーがコンテンツに対する認可を持っているかどうかを決定します。内部的に、検索エンジンはセキュリティプリンシパルに関連付けられていないドキュメントをフィルタリングします。

この機能は現在プレビュー段階です。

インデクサーとプルAPIを使用した代替アプローチについては、[Quickstart-Document-Permissions-Pull-API](../Quickstart-Document-Permissions-Pull-API/document-permissions-pull-api.ipynb)を参照してください。


## 前提条件

+ Azure AI Search、[ロールベースのアクセス制御](https://learn.microsoft.com/azure/search/search-security-enable-roles)を持つもの。

## 権限

このチュートリアルではMicrosoft Entra ID認証と認可を使用します。

Azure AI Searchでは、オブジェクトを作成してクエリを実行するためにロール割り当てが必要です：

+ **Search Service Contributor**
+ **Search Index Data Contributor**
+ **Search Index Data Reader**

詳細については、[ロールを使用してAzure AI Searchに接続する](https://learn.microsoft.com/azure/search/search-security-rbac)と[クイックスタート: ローカルテスト用にキーなしで接続する](https://learn.microsoft.com/azure/search/search-get-started-rbac)を参照してください。

## 環境変数の設定

1. `sample.env`を`.env`にリネームします。
1. `.env`ファイルで、検索サービスへの完全なエンドポイント（https://your-search-service.search.windows.net）を提供します。
1. 異なる名前を希望する場合は、デフォルトのインデックス名を置き換えます。

## 接続の読み込み

このサンプルコードを実行するには、仮想環境を作成することをお勧めします。Visual Studio Codeで、コントロールパレット（Ctrl+Shift+P）を開いて環境を作成してください。このノートブックはPython 3.10でテストされています。

環境が作成されたら、環境変数を読み込みます。

In [ ]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import os

load_dotenv(override=True) # take environment variables from .env.

# The following variables from your .env file are used in this notebook
endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
credential = DefaultAzureCredential()
index_name = os.getenv("AZURE_SEARCH_INDEX")
token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")


## サンプルインデックスの作成

検索インデックスには、コンテンツと権限メタデータのフィールドを含める必要があります。新しい権限フィルターオプションを文字列フィールドに割り当て、フィールドがフィルタリング可能であることを確認してください。検索エンジンはクエリ時に内部的にフィルターを構築します。

In [ ]:
from azure.search.documents.indexes.models import SearchField, SearchIndex, PermissionFilter, SearchIndexPermissionFilterOption
from azure.search.documents.indexes import SearchIndexClient

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index = SearchIndex(
    name=index_name,
    fields=[
        SearchField(name="id", type="Edm.String", key=True, filterable=True, sortable=True),
        SearchField(name="oid", type="Collection(Edm.String)", retrievable=True, filterable=True, permission_filter=PermissionFilter.USER_IDS),
        SearchField(name="group", type="Collection(Edm.String)", retrievable=True, filterable=True, permission_filter=PermissionFilter.GROUP_IDS),
        SearchField(name="name", type="Edm.String", searchable=True)
    ],
    permission_filter_option=SearchIndexPermissionFilterOption.ENABLED
)

index_client.create_index(index=index)
print(f"Index '{index_name}' created with permission filter option enabled.")

## Graphに接続してオブジェクトID（OID）とグループを見つける

このステップでは、Graph APIを呼び出してMicrosoft Entra IDのグループIDをいくつか取得します。あなたのグループIDは、次のステップで作成されるオブジェクトのアクセス制御リストに追加されます。

In [ ]:
from msgraph import GraphServiceClient
client = GraphServiceClient(credentials=credential, scopes=["https://graph.microsoft.com/.default"])

groups = await client.me.member_of.get()
me = await client.me.get()
oid = me.id

## サンプルデータのアップロード

このステップでは、コンテナー、フォルダーを作成し、ドキュメントをAzure Storageにアップロードします。各ファイルのアクセス制御リストにあなたのグループIDを割り当てます。

In [ ]:
from azure.search.documents import SearchClient
search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential)

documents = [
    { "id": "1", "oid": [oid], "group": [groups.value[0].id], "name": "Document 1" },
    { "id": "2", "oid": ["all"], "group": [groups.value[0].id], "name": "Document 2" },
    { "id": "3", "oid": [oid], "group": ["all"], "name": "Document 3" },
    { "id": "4", "oid": ["none"], "group": ["none"], "name": "Document 4" },
    { "id": "5", "oid": ["none"], "group": [groups.value[0].id], "name": "Document 5" },
]
search_client.upload_documents(documents=documents)
print("Documents uploaded to the index.")


## x-ms-query-source-authorizationを使用したサンプルデータの検索

このクエリでは、無条件検索を提供するために空の検索文字列（`*`）を使用します。ファイル名と各ファイルに関連付けられた権限メタデータを返します。各ファイルが異なるグループIDに関連付けられていることに注意してください。

In [ ]:
results = search_client.search(search_text="*", x_ms_query_source_authorization=token_provider(), select="name,oid,group", order_by="id asc")

for result in results:
    print(f"Name: {result['name']}, OID: {result['oid']}, Group: {result['group']}")

## x-ms-query-source-authorizationなしでのサンプルデータの検索

このステップでは、認証が失敗した場合のユーザーエクスペリエンスを示します。レスポンスには結果が返されません。

In [ ]:
results = search_client.search(search_text="*", x_ms_query_source_authorization=None, select="name,oid,group", order_by="id asc")

for result in results:
    print(f"Name: {result['name']}, OID: {result['oid']}, Group: {result['group']}")

## 次のステップ

詳細については、[Azure AI Searchでのドキュメントレベルアクセス制御](https://learn.microsoft.com/azure/search/search-document-level-access-overview)を参照してください。